In [16]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [3]:
import pandas as pd

In [4]:
import kss
import re

In [5]:
from sklearn.neighbors import NearestNeighbors

In [6]:
model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_sts'
model = SentenceTransformer(model_path)
#model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


/home/u4026/.local/lib/python3.8/site-packages/sentence_transformers/models/Transformer.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.auto_model.load_state_dict(

In [7]:
nli_model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_nli'
nli_model = SentenceTransformer(nli_model_path)

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


In [28]:
premise = "모든 고양이는 동물이다."
hypothesis = "고양이는 동물이다."

embeddings = model.encode([premise, hypothesis], convert_to_tensor=True)
cosine_score = util.pytorch_cos_sim(embeddings[0], embeddings[1])

print(f"유사도 점수: {cosine_score.item()}")

유사도 점수: 0.8977875709533691


In [4]:
pip show transformers

Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/u4026/.local/lib/python3.8/site-packages
Requires: regex, filelock, pyyaml, tokenizers, huggingface-hub, safetensors, requests, numpy, tqdm, packaging
Required-by: sentence-transformers, kobert
Note: you may need to restart the kernel to use updated packages.


In [8]:
# CSV 파일 로드
csv_file = "books_summary.csv"
data = pd.read_csv(csv_file)

In [25]:
def split_and_filter_sentences(text):
    """
    텍스트를 KSS와 추가 기준으로 분리하고, 의미 없는 연결어를 제거
    """
    # KSS로 1차 분리
    sentences = kss.split_sentences(text)
    
    # 추가 기준 적용 ('그리고', '또', ',' 기준으로 분리)
    split_sentences = []
    for sentence in sentences:
        parts = re.split(r'(그리고|또|,)', sentence)
        parts = [p.strip() for p in parts if p.strip()]  # 공백 제거 및 빈 문자열 제거
        split_sentences.extend(parts)
    
    # 불필요한 연결어 필터링
    filtered_sentences = [s for s in split_sentences if s not in ["그리고", "또", ","]]
    
    return filtered_sentences

In [21]:
def get_average_embedding(sentences, model):
    """
    여러 문장의 임베딩 값을 평균화
    """
    if not sentences:
        return None  # 빈 입력에 대한 처리

    embeddings = model.encode(sentences, convert_to_tensor=True)
    avg_embedding = embeddings.mean(dim=0)  # 평균 계산
    return avg_embedding

In [9]:
# 전처리된 텍스트 열 추출
corpus = data['내용요약'].tolist()
titles = data['첵제목'].tolist()

In [26]:
# 코퍼스 문장 분리 및 임베딩 평균 계산
corpus_embeddings = []
for doc in corpus:
    sentences = split_and_filter_sentences(doc)
    avg_embedding = get_average_embedding(sentences, model)
    corpus_embeddings.append(avg_embedding)

In [10]:
# 코퍼스 임베딩 생성
corpus_embeddings = nli_model.encode(corpus, convert_to_tensor=True)

In [14]:
# 사용자가 입력한 텍스트
query = input("텍스트를 입력하세요: ")
#query_sentences = split_and_filter_sentences(query)
#query_embedding = get_average_embedding(query_sentences, model)

# 쿼리 임베딩 생성
query_embedding = nli_model.encode([query], convert_to_tensor=True)

텍스트를 입력하세요: 독자를 글쓰기의 생산자로 전환하는 과정이 교육적으로 어떤 영향을 미치는지 알려주는 책 추천해줘


In [28]:
# 코사인 유사도 비교
if query_embedding is not None:
    cos_scores = [util.pytorch_cos_sim(query_embedding, corpus_embedding)[0].item() for corpus_embedding in corpus_embeddings]

    # 상위 유사도 결과 출력
    top_k = min(5, len(cos_scores))
    top_results = np.argsort(cos_scores)[-top_k:][::-1]  # 유사도 높은 순으로 정렬

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop {} most similar sentences in corpus:".format(top_k))

    for idx in top_results:
        print(f"{corpus[0]} (Score: {cos_scores[idx]:.4f})")
else:
    print("입력 텍스트에서 유효한 문장이 없습니다.")





Query: 요즘 우리나라 대통령이 개엄령을 일으켜서 놀라웠는데 예전에 있었던 광주시위에 관한책을 소개시켜줘 광주시위후 슬픔을 받는 사람들의 이야기였으면 해

Top 3 most similar sentences in corpus:
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.4742)
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.2166)
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.1448)


In [15]:
# 코사인 유사도 계산
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()

# 책 제목별 최고 유사도 선택
best_results = {}
for idx, score in enumerate(cos_scores):
    title = titles[idx]
    sentence = corpus[idx].strip()
    if title not in best_results or best_results[title]['score'] < score.item():
        best_results[title] = {'sentence': sentence, 'score': score.item()}

    # 유사도 점수가 높은 순으로 정렬
sorted_results = sorted(best_results.items(), key=lambda x: x[1]['score'], reverse=True)

# 가장 유사한 결과 가져오기
#top_k = 3
#top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
top_k=3
# 결과 출력
print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")
for title, result in sorted_results[:top_k]:
        print(f"Title: {title}, Sentence: {result['sentence']} (Score: {result['score']:.4f})")

#for idx in top_results[0:top_k]:
    #print(f"Title: {titles[idx]}, Sentence: {corpus[idx].strip()} (Score: {cos_scores[idx]:.4f})")
    #print(f"{corpus[idx].strip()} (Score: {cos_scores[idx]:.4f})")





Query: 독자를 글쓰기의 생산자로 전환하는 과정이 교육적으로 어떤 영향을 미치는지 알려주는 책 추천해줘

Top 5 most similar sentences in corpus:
Title: 춘향전의 수용문화, Sentence: 바르뜨는 독자가 글쓰기의 즐거움, 텍스트를 생산하는 즐거움을 갖게 하기 위해서 독자를 소비자에서 생산자로 전환하는 관계 재정립이 필요하다고 하였다. 문학 생산자와 소비자의 명확한 구분을 없애면, 아마추어에게 쓰기의 즐거움을 알려주고 창조적 과정에 접근하도록 만들 수 있다. 개작 텍스트의 생산은 학습 주체의 쓰기 활동에 활용할 수 있으며, 이를 통해 학습 주체가 읽고 쓰는 능력인 문해력(literacy)을 얻게 하는 것이 국어교육의 목표가 되어야 한다. (Score: 0.9768)
Title: 국제 교육 모니터링 체제 구축을 위한 학습성과 지표 개발 연구 -수학·읽기 영역의 학습성가 명세화 및 평가 방안, Sentence: O'Donnell & Wood와 Jett-Simpson 등은 읽기의 단계에 대해 설명하였으며, 이를 '초기 읽기', '전이 읽기', '통합적 독자', '완숙기 독자', '기초 문식성' 등의 단계로 표현했다. 소리와 글자의 대응, 해석, 유창성, 추론 등이 각 단계를 구분하는 기준이 되기도 하며, 읽기 과정의 자동화, 목적에 따른 읽기 등의 특징을 갖기도 한다. (Score: 0.9253)
Title: 미래 사회 교육 환경 변화에 따른 교과서 발전 방안, Sentence: 함수곤은 전통적인 학습 교재로서의 교과서의 역할을 다섯 가지 제시했다. 또한 학습 과제의 효율적인 제시, 학습의 개성화 기능, 정착 기능 등을 최근 등장한 교과서의 학습재로서의 역할을 위해 필요하다고 설명했다. (Score: 0.9175)


In [6]:
corpus = ['갑자기 테이블 바깥쪽에 앉아있던 피해자 C(여, 39세)의 왼쪽 가슴을 3회 만져 피해자를 강제로 추행하였다.',
          '주거지에 찾아와 "같이 살자"라고 말을 하며 피해자의 상의를 걷어 올려 양손으로 피해자의 양쪽 가슴을 수차례 주무른 후 피해자의 손을 잡아채 피고인의 성기에 가져가 피고인의 성기를 만지게 하고, 이어 피해자의 바지 위에 손을 얹어 피해자의 음부를 만졌다.\\n2. 2020. 8. 8.자 범행\\n피고인은 2020. 8. 8. 09:00경 제1항의 피해자 주거지에서, 마당에 있다가 집 안으로 들어가는 피해자를 따라 들어가 피해자의 상의 옷 안으로 한 손을 넣어 가슴을 주무르고, 피해자의 바지 속에 손을 넣어 피해자의 음부를 만졌다.\\n이로써 피고인은 피해자를 강제로 추행하였다.',
          "피해자 D(가명)에게 다가가 ‘자기야'라고 말하면서 피해자 왼편에 서서 오른 팔로 피해자의 어깨를 감싸 안아 피해자를 강제로 추행하였다.",
          '피고인은 종업원인 피해자 C(여, 21세)와 이야기를 하면서 갑자기 손으로 피해자의 가슴을 3회 가량 쳐 피해자를 강제로 추행하였다.',
          "피해자에게 ‘한번 하자, 안 해주면 안 간다. 아는 사람을 시켜서 장사 못하게 한다.'라는 등의 말을 하면서 성관계를 요구하였으나 피해자가 이를 거절하자 갑자기 피해자의 엉덩이와 가슴을 손으로 수회 움켜잡고, 이를 뿌리치고 일어나는 피해자에게 ‘한번 안아주면 가겠다.'라고 말하여 이에 피해자가 어쩔 수 없이 응하자 그 틈을 타 피해자의 팬티 안으로 손을 집어넣어 피해자의 음부를 수회 움켜쥐는 등 피해자를 강제로 추행하였다."
          '피고인과 피해자 B(여, 37세)는 학교 선후배 사이로 알고 지내는 관계이다. 식당의 1, 2층을 연결하는 내부 계단 중간 부분에서, 피해자와 마주치자 갑자기 손으로 피해자의 허리를 끌어당겨 안으면서 피해자에게 "이러면 내가 따먹는다"라고 말하여 피해자를 강제로 추행하였다.',
          '차량 안에서 갑자기 피해자의 머리를 세게 잡아당겨 입술에 키스하였고, 같은 날 23:50경 같은 시에 있는 번지불상의 도로를 진행하는 위 차량 안에서 운전 중인 피해자의 가슴을 만지려고 시도하여 피해자가 피고인의 손을 쳐내면서 "뭐하는 거냐, 술집 여자도 아닌데?"라고 항의하는데도 "가슴이 예쁘다, 하나도 안 꺼졌다"라고 말하면서 피해자의 블라우스 안으로 손을 집어넣고 가슴을 주물렀다.거부하는데도 "나랑 만나는 거 생각해 봐, 남편한테 너를 빼앗아 오겠다, 익산에 가서 먹겠다"라고 말하면서 피해자의 블라우스 안으로 손을 집어 넣고 가슴을 여러 번 주물렀다.하차하려는 피해자를 제지하면서 머리를 잡아당겨 입술에 키스하고, 피해자의 블라우스를 세게 잡아당겨 단추가 뜯어지자 피해자의 가슴을 입으로 빨고, "모텔에 같이 가자"고 말하면서 피해자의 바지 후크를 풀어 지퍼를 내리고 손을 집어넣어 피해자의 음부를 만졌다.\\n이로써 피고인은 피해자를 추행하였다.',
          '피고인의 집에서, 피고인의 부탁으로 기타를 가르쳐 주러 온 피해자가 피고인의 방침대에 걸터앉아 휴대폰 충전을 하고 있는 것을 보고 피해자의 뒤로 다가가, 갑자기 양팔로 피해자의 허리를 껴안으면서 "나 섹스하고 싶어"라고 말하고, 이에 피해자가 피고인의 손을 뿌리치자, 양팔을 빼는 과정에서 갑자기 손으로 피해자의 허벅지 부위를 쓸어내리듯이 만지고, 이에 피해자가 피고인을 피하여 자리에서 일어나자, 피해자의 손목을 잡아 피해자를 침대에 다시 앉힌 다음 피해자의 뒤에서 갑자기 양팔로 피해자의 허리를 껴안으면서 피해자에게 "진짜 한번만 해 주면 안 돼?"라고 말하고, 계속하여 피고인을 피하여 자리에서 일어나 나가려고 하는 피해자의 팔을 끌어 당겨 피해자가 침대 앞에 있는 피아노 의자에 앉자, 피해자에게 "너 손 작다."라고 말하면서 갑자기 피해자의 손에 깍지를 끼고, 이에 또다시 피해자가 손을 뺐다. 계속하여 피고인은 피해자에게"나 진짜 못 참겠으니까 한번만 하자"라고 하면서 갑자기 피해자의 손목을 잡아당겼다.\\n이로써 피고인은 피해자를 강제로 추행하였다.',
          "술을 마시던 중 자리에서 일어나 피해자에게 다가가 오른손을 피해자 옷 속에 집어넣어 왼쪽 가슴을 4회 만졌다.이로써 피고인은 피해자를 강제로 추행하였다.",
          '피고인의 옆에서 수영 강습을 받기 위해 물에 엎드린 자세로 떠 있던 피해자의 엉덩이를 갑자기 손으로 움켜쥐었다.이로써 피고인은 피해자를 강제로 추행하였다.']

corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

query = ['피고인은 엉덩이를 손으로 움켜쥐어 피해자를 강제로 추행하였다.']

query_embedding = model.encode(query, convert_to_tensor=True)

cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()

top_k = 5

top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")

for idx in top_results[0:top_k]:
  print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: ['피고인은 엉덩이를 손으로 움켜쥐어 피해자를 강제로 추행하였다.']

Top 5 most similar sentences in corpus:
피고인의 옆에서 수영 강습을 받기 위해 물에 엎드린 자세로 떠 있던 피해자의 엉덩이를 갑자기 손으로 움켜쥐었다.이로써 피고인은 피해자를 강제로 추행하였다. (Score: 0.7840)
피해자 D(가명)에게 다가가 ‘자기야'라고 말하면서 피해자 왼편에 서서 오른 팔로 피해자의 어깨를 감싸 안아 피해자를 강제로 추행하였다. (Score: 0.7435)
술을 마시던 중 자리에서 일어나 피해자에게 다가가 오른손을 피해자 옷 속에 집어넣어 왼쪽 가슴을 4회 만졌다.이로써 피고인은 피해자를 강제로 추행하였다. (Score: 0.7283)
갑자기 테이블 바깥쪽에 앉아있던 피해자 C(여, 39세)의 왼쪽 가슴을 3회 만져 피해자를 강제로 추행하였다. (Score: 0.7032)
피고인은 종업원인 피해자 C(여, 21세)와 이야기를 하면서 갑자기 손으로 피해자의 가슴을 3회 가량 쳐 피해자를 강제로 추행하였다. (Score: 0.7030)
